## Asteroid conjunction finder

Find close approaches between asteroid positions (obtained dynamically from MPC ephemeris service) and NGC DSO catalog (cached locally from Vizier table)

In [2]:
# Parameters, edit to taste

# Date & time for ephemeris lookup

OBS_TIME = '2025-01-13 20:00:00'

# coords of home location (used to calculate altitude of conjunction)

OBS_LAT = 52.9
OBS_LONG = 0.5

# minimum altitude for conjunctions (anything below is filtered out for visibility)
MIN_VISIBLE_ALT = 10 # degrees

# mimimum angular size of DSO to be visible in image
DSO_MIN_SIZE = 2  # 5% of Seestar S50 FoV width

# maximum magnitude for DSO to be visible 
DSO_MAX_MAG = 15  # approx limiting magnitude for Seestar S50

# Max angular distance between asteroid and DSO
MAX_SEPARATION_ARCMIN = 40 # approx frame width in Seestar S50

# number of asteroids to check (from list ordered by absolute magnitude)
CHECK_ASTEROIDS = 100

In [8]:
# load list of asteroids to search from file (ordered by absolute magnitude)

import csv

with open('./asteroids with abs mag below 10.csv') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader) # skip header row
    asteroid_names = [row[3] for _, row in zip(range(CHECK_ASTEROIDS), csv_reader)]

In [ ]:
# Fetch ngc catalog from CDS vizier server
# This code only needs to be run once to cache the table locally

# from astroquery.vizier import Vizier

# vizier = Vizier()
# vizier.ROW_LIMIT = -1

# ngc_tables = vizier.get_catalogs('VII/118/ngc2000')

# # save ngc table locally for future reference

# ngc_tables[0].write('./ngc_catalog.ecsv')


In [10]:
# create SkyCoord catalog of ngc DSOs for searching

from astropy.io import ascii

# load locally cached ngc table 
ngc_table = ascii.read('./ngc_catalog.ecsv')

# filter table to size and magnitude limits

filtered_ngc_table = ngc_table[ngc_table['size'] > DSO_MIN_SIZE]
filtered_ngc_table = filtered_ngc_table[filtered_ngc_table['mag'] < DSO_MAX_MAG]

In [11]:
from astropy.coordinates import SkyCoord
import astropy.units as u

ngc_catalog = SkyCoord(ra = filtered_ngc_table['RAB2000'], dec = filtered_ngc_table['DEB2000'],
                       unit = (u.hourangle, u.deg))



In [12]:
# Query MPC Horizons service for asteroid ephemeris data

from astroquery.mpc import MPC
from astropy.table import vstack

ephemerides = [MPC.get_ephemeris(a, start = OBS_TIME, number = 1) for a in asteroid_names]
ephemerides = vstack(ephemerides)

# create SkyCoord catalog for search
ephemerides_cat = SkyCoord(ra = ephemerides['RA'], dec = ephemerides['Dec'], unit = u.deg)

In [29]:
# Find sky conjunctions of asteroid positions and NGC catalog

from astropy.coordinates import AltAz, EarthLocation

import warnings
warnings.filterwarnings('ignore') # block formatting warning message

idx_ngc, idx_asteroid, sep2d, _ = ephemerides_cat.search_around_sky(ngc_catalog, seplimit = MAX_SEPARATION_ARCMIN * u.arcmin)

home = EarthLocation(lat = OBS_LAT * u.deg, lon = OBS_LONG * u.deg)
home_altaz = AltAz(obstime = OBS_TIME, location = home)

print('Predicted conjunctions for ', OBS_TIME, '\n')
for ingc, ia, sep in zip(idx_ngc, idx_asteroid, sep2d):
    ngc = filtered_ngc_table[ingc]
    ngc_loc = ngc_catalog[ingc]
    ephem = ephemerides_cat[ia]
    ngc_mag = filtered_ngc_table[ingc]['mag']
    ast_mag = ephemerides[ia]['V']
    altaz_ast =  ephem.transform_to(home_altaz).alt.degree
    altaz_ngc = ngc_loc.transform_to(home_altaz).alt.degree
    skip_count = 0
    name = ngc['Name']
    if name[0] != 'I':
        name = 'NGC ' + name
    if altaz_ast < MIN_VISIBLE_ALT or altaz_ngc < MIN_VISIBLE_ALT:
        skip_count += 1
        continue
    print(f"{name}, {ngc['Type']}, (alt={altaz_ngc:.1f}, mag={ngc_mag:.1f}) <> {asteroid_names[ia]} (alt={altaz_ast:.1f}, mag={ast_mag:.1f}), sep = {sep:.2f}")
print('\nThere were', skip_count, 'conjunctions below the visibility threshold')
    

Predicted conjunctions for  2025-01-13 20:00:00 

NGC 428, Gx, (alt=30.4, mag=11.4) <> Camilla (alt=30.2, mag=13.4), sep = 0.49 deg
NGC 514, Gx, (alt=42.4, mag=11.9) <> Hygiea (alt=42.5, mag=11.8), sep = 0.37 deg
NGC 936, Gx, (alt=34.3, mag=10.1) <> Davida (alt=34.8, mag=11.3), sep = 0.64 deg
NGC 941, Gx, (alt=34.3, mag=12.4) <> Davida (alt=34.8, mag=11.3), sep = 0.53 deg
NGC 955, Gx, (alt=34.4, mag=12.0) <> Davida (alt=34.8, mag=11.3), sep = 0.60 deg
I 443, Nb, (alt=47.0, mag=--) <> Thisbe (alt=47.3, mag=11.9), sep = 0.35 deg

There were 1 conjunctions below the visibility threshold
